In [1]:
from sklearn.model_selection import KFold,StratifiedKFold,GroupKFold
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm_notebook, tnrange
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import datetime
import gc
import lightgbm as lgb
import os
DATA_PATH = './datasets/'
warnings.filterwarnings("ignore")
%matplotlib inline

In [14]:
df_train = pd.read_csv(DATA_PATH+'train.csv')
df_test = pd.read_csv(DATA_PATH+'test.csv')

In [27]:
def modelKFoldReg(df_train,df_test,model):
    
    NFOLDS = 5
    kfold = KFold(n_splits=NFOLDS,shuffle=True,random_state=2018)
    ntrain = df_train.shape[0]
    ntest = df_test.shape[0]

#     获取预测叶子节点Index
    train_index_features = np.zeros((ntrain,model.n_estimators))
    test_index_features = np.zeros((ntest,model.n_estimators))
    
    for foldIndex,(dev_index,val_index) in enumerate(kfold.split(df_train)):
        x_dev = df_train[tr_features].iloc[dev_index]
        y_dev = df_train[label].iloc[dev_index]
        x_val = df_train[tr_features].iloc[val_index]
        y_val = df_train[label].iloc[val_index]
        model.fit(x_dev, y_dev,eval_set=[(x_dev,y_dev),(x_val,y_val)],early_stopping_rounds=50,verbose=100)
    
    train_index_features = model.predict(df_train[tr_features],pred_leaf=True,num_iteration=model.best_iteration_)
    test_index_features = model.predict(df_test[tr_features],pred_leaf=True,num_iteration=model.best_iteration_)
    
    return model,train_index_features,test_index_features

label = ['target']
dropCols = ['card_id','first_active_month','target','is_outlier']
tr_features = [_f for _f in df_train.columns if _f not in dropCols and df_train[_f].dtype!='object']

lgb_params={
    'learning_rate': 0.01,
    'objective':'regression',
    'n_estimators':10,
    'metric':'rmse',
    'num_leaves': 10,
    'verbose': 1,
    "random_state":2018,
    'device': 'gpu',
    'gpu_platform_id':1,
    'gpu_device_id': 1,
}
lgb_est = lgb.LGBMRegressor(**lgb_params)

lgb_est,train_index_features,test_index_features = modelKFoldReg(df_train,df_test,lgb_est)


Training until validation scores don't improve for 50 rounds.
Did not meet early stopping. Best iteration is:
[10]	training's rmse: 3.84866	valid_1's rmse: 3.85637
Training until validation scores don't improve for 50 rounds.
Did not meet early stopping. Best iteration is:
[10]	training's rmse: 3.86154	valid_1's rmse: 3.8045
Training until validation scores don't improve for 50 rounds.
Did not meet early stopping. Best iteration is:
[10]	training's rmse: 3.85016	valid_1's rmse: 3.85032
Training until validation scores don't improve for 50 rounds.
Did not meet early stopping. Best iteration is:
[10]	training's rmse: 3.85262	valid_1's rmse: 3.84044
Training until validation scores don't improve for 50 rounds.
Did not meet early stopping. Best iteration is:
[10]	training's rmse: 3.83792	valid_1's rmse: 3.89886


In [33]:
df_feature_train = pd.DataFrame(data=train_index_features)
df_feature_train.columns=['featureIndex_%s'%index for index in range(10)]
df_feature_train['card_id'] = df_train.card_id.values
df_feature_train.to_csv('./datasets/df_feature_train.csv',index=False)

df_feature_test = pd.DataFrame(data=test_index_features)
df_feature_test.columns=['featureIndex_%s'%index for index in range(10)]
df_feature_test['card_id'] = df_test.card_id.values
df_feature_test.to_csv('./datasets/df_feature_test.csv',index=False)